In [76]:
# STEP 0: Must be done before anything each time notebook is loaded
# Import PySplit
import pysplit as py
import numpy as np
import pickle as pickle
from pprint import pprint
import os
import fnmatch

In [79]:

#directory setup#direct 
working_dir = r'C:/hysplit4/working'
storage_dir = r'C:/hysplit4/pysplit/sumatra'
meteo_dir = r'E:\nceptest2p3'
# meteo_dir = r'C:\hysplit4\gdas'
meteoyr_2digits = False
meteo_bookends=([0], [0])

In [80]:
os.listdir(meteo_dir)

['renamerpmonYYYY_n.sh',
 'RPapr2010_0',
 'RPaug2010_0',
 'RPdec2010_0',
 'RPfeb2010_0',
 'RPjan2010_0',
 'RPjul2010_0',
 'RPjun2010_0',
 'RPmar2010_0',
 'RPmay2010_0',
 'RPnov2010_0',
 'RPoct2010_0',
 'RPsep2010_0']

In [81]:
# STEP 1: TRAJECTORY GENERATION (not yet tested on mac)
# check out bulk_trajen_example.py

#file basename
basename = 'sumatratest'

#Arguments
years = [2010]
# years=range(1979,2019)
months = [10]
# months=range(1,13)
hours = [10]
altitudes = [1100]
location = (5.36, 100.76)
runtime = -48

In [82]:
def _mondict(n_hem=True):
    if n_hem:
        season_month_dict = {12: ['winter', 'dec'],
                             1 : ['winter', 'jan'],
                             2 : ['winter', 'feb'],
                             3 : ['spring', 'mar'],
                             4 : ['spring', 'apr'],
                             5 : ['spring', 'may'],
                             6 : ['summer', 'jun'],
                             7 : ['summer', 'jul'],
                             8 : ['summer', 'aug'],
                             9 : ['autumn', 'sep'],
                             10: ['autumn', 'oct'],
                             11: ['autumn', 'nov']}
    else:
        season_month_dict = {12: ['summer', 'dec'],
                             1 : ['summer', 'jan'],
                             2 : ['summer', 'feb'],
                             3 : ['autumn', 'mar'],
                             4 : ['autumn', 'apr'],
                             5 : ['autumn', 'may'],
                             6 : ['winter', 'jun'],
                             7 : ['winter', 'jul'],
                             8 : ['winter', 'aug'],
                             9 : ['spring', 'sep'],
                             10: ['spring', 'oct'],
                             11: ['spring', 'nov']}

    return season_month_dict

In [83]:
# Initialize dictionary of months, seasons
n_hemisphere = True
if location[0] < 0:
    n_hemisphere = False

mon_dict = _mondict(n_hem=n_hemisphere)


In [84]:
    # Get the strings that will match files for the previous, next,
    # and current months
   # Set year formatting in 3 places
def _year2string(year):
        return '{0:02}'.format(year % 100)
    meteoyr_2digits = False
    yr_is2digits = {True : _year2string,
                    False : str}
    
    controlyearfunc = yr_is2digits[True]
    meteoyearfunc = yr_is2digits[meteoyr_2digits]

In [96]:
mon=6
year=2010
def _monyearstrings(mon, year, mon_dict, meteoyearfunc):
        next_year = year
        prev_year = year

        next_mon = mon + 1
        prev_mon = mon - 1

        if prev_mon == 0:
            prev_mon = 12
            prev_year = year - 1
        if next_mon == 13:
            next_mon = 1
            next_year = year + 1

        w = '*'

        prv = w + mon_dict[prev_mon][1] + w + meteoyearfunc(prev_year) + w
        nxt = w + mon_dict[next_mon][1] + w + meteoyearfunc(next_year) + w
        now = w + mon_dict[mon][1] + w + meteoyearfunc(year) + w

        return prv, nxt, now

In [91]:
mon=6
year=2010
def _meteofinder(meteo_dir, meteo_bookends, mon, year, mon_dict,
                 meteoyearfunc):
#     """
#     Get list of meteorology files.
#     Creates list of files in storage location ``meteo_dir`` that belong
#     to the given month and year, plus the necessary files from previous
#     and the next months (``meteo_bookends``).
#     For successful meteofinding, separate different meteorology types into
#     different folders and name weekly or semi-monthly files according to the
#     following convention:
#         *mon*YY*#
#     where the * represents a Bash wildcard.
#     Parameters
#     ----------
#     meteo_dir : string
#         Full or relative path to the location of the meteorology files.
#     meteo_bookends : tuple of lists of ints
#         To calculate a month of trajectories, files from the previous and next
#         month must be included.  This indicates which file numbers from the
#         previous month and which from the next month are necessary.
#         The user is responsible for making sure the correct bookends for their
#         trajectory length and meteorology file periods are provided.
#     mon : int
#         The integer representation of the current month.  Converted to a
#         3-letter string to find meteorology files.
#     year : int
#         The integer representation of the current year.  Converted to a length
#         2 string to find meteorology files.
#     mon_dict : dictionary
#         Dictionary keyed by month integer, with lists of [season, mon]
#     meteoyearfunc : function
#         Function that formats the year string to length 2 or 4 to identify
#         appropriate meteorology files
#     Returns
#     -------
#     meteofiles : list of strings
#         List of strings representing the names of the required
#         meteorology files
#     """
    # Current working directory set in generate_bulktraj() environment
    orig_dir = os.getcwd()

    # Initialize lists, count
    meteofiles = []
    file_number = -1

    # Get the strings that will match files for the previous, next,
    # and current months
    prv, nxt, now = _monyearstrings(mon, year, mon_dict, meteoyearfunc)

    # Change directory and walk through files
    try:
        os.chdir(meteo_dir)

        _, _, files = next(os.walk('.'))

        # Order of files to CONTROL doesn't matter
        for each_file in files:
            if fnmatch.fnmatch(each_file, now):
                meteofiles.append(each_file)
            elif fnmatch.fnmatch(each_file, prv):
                if int(each_file[file_number]) in meteo_bookends[0]:
                    meteofiles.append(each_file)
            elif fnmatch.fnmatch(each_file, nxt):
                if int(each_file[file_number]) in meteo_bookends[1]:
                    meteofiles.append(each_file)

    finally:
        os.chdir(orig_dir)

    num_files = len(meteofiles)

    if num_files == 0:
        raise OSError('0 files found for month/year %(mon)d / %(year)d'
                      %{'mon': mon, 'year': year})
        
    if num_files > 12:
        print(meteofiles)
        raise OSError('%(f)d files found for month/year %(mon)d / %(year)d.'\
                      '  Maximum 12 allowed.  If wrong years are included, '\
                      'identify files by 4 digit years (meteoyr_2digits=True).'\
                      '  May require renaming meteorology files.'
                      %{'f': num_files, 'mon': mon, 'year': year})

    return meteofiles


In [103]:
 # Assemble list of meteorology files
m=6
y=2010

meteofiles = _meteofinder(meteo_dir, meteo_bookends, m, y,
                          mon_dict, meteoyearfunc)
meteofiles

['RPjul2010_0', 'RPjun2010_0', 'RPmay2010_0']

In [101]:
prv, nxt, now = _monyearstrings(mon, year, mon_dict, meteoyearfunc)
prv,nxt,now


TypeError: can only concatenate list (not "int") to list

In [88]:
each_file[-1]

'l'

In [89]:

os.chdir(meteo_dir)

_, _, files = next(os.walk('.'))
file_number=-1
#         Order of files to CONTROL doesn't matter
for each_file in files:
    print(each_file[-1])
    if fnmatch.fnmatch(each_file, now):
        print(each_file)

    elif fnmatch.fnmatch(each_file, prv):
        if int(each_file[file_number]) in meteo_bookends[0]:
            print(each_file)
            
    elif fnmatch.fnmatch(each_file, nxt):
        if int(each_file[file_number]) in meteo_bookends[1]:
            print(each_file)

h
0
0
0
0
0
0
RPjul2010_0
0
RPjun2010_0
0
0
RPmay2010_0
0
0
0


In [44]:
files

['renamerpmonYYYY_n.sh',
 'RPapr2010_0.gbl',
 'RPaug2010_0.gbl',
 'RPfeb2010_0.gbl',
 'RPjan2010_0.gbl',
 'RPjul2010_0.gbl',
 'RPjun2010_0.gbl',
 'RPmar2010_0.gbl',
 'RPmay2010_0.gbl',
 'RPnov2010_0.gbl',
 'RPoct2010_0.gbl',
 'RPsep2010_0.gbl']

In [5]:
#Generate trajs
py.generate_bulktraj(basename, working_dir, storage_dir, meteo_dir,
                          years, months, hours, altitudes, location, runtime, meteoyr_2digits=False,
                          monthslice=slice(0, 32, 5), meteo_bookends=([4, 5], [1]))

ValueError: invalid literal for int() with base 10: 'l'

In [61]:
meteo_bookends[1]

[1]